# 🚀 09. 대용량 처리 & 실전 종합 심화

> **목표**: 지금까지 배운 모든 것을 합쳐 **실무 수준의 자동화 시스템**을 구축한다.

---

### 📚 이 챕터에서 배우는 것

| 순서 | 주제 | 핵심 기술 |
|:---:|:---|:---|
| 1 | 대용량 데이터 처리 | pandas 청크, 메모리 최적화 |
| 2 | 멀티파일 자동 통합 | glob, pathlib, 반복 자동화 |
| 3 | 동적 대시보드 생성 | xlwings 차트 + openpyxl 서식 |
| 4 | 이메일 자동 발송 | smtplib + Excel 첨부 |
| 5 | 예외 처리 & 로깅 | try/except, logging 모듈 |
| 6 | 🔥 실전 종합 | 월간 영업 보고서 자동화 시스템 |

---
## 1️⃣ 대용량 데이터 처리

> 수만~수십만 행 데이터를 **메모리 효율적으로** 처리하는 방법

### 핵심 개념
- **청크(Chunk) 처리**: 한 번에 전체를 읽지 않고 나눠서 처리
- **dtype 최적화**: 불필요한 메모리 사용 줄이기
- **usecols**: 필요한 열만 읽기

In [ ]:
# 실습용 대용량 더미 데이터 생성 (10만 행)
import pandas as pd
import numpy as np
import random
from pathlib import Path

random.seed(42)
np.random.seed(42)

N = 100_000
departments = ['영업팀', '마케팅팀', '개발팀', '인사팀', '재무팀']
products    = ['노트북', '마우스', '키보드', '모니터', '헤드셋']
regions     = ['서울', '부산', '대구', '인천', '광주']

df_big = pd.DataFrame({
    '날짜':     pd.date_range('2024-01-01', periods=N, freq='1min').strftime('%Y-%m-%d'),
    '부서':     np.random.choice(departments, N),
    '담당자':   [f'직원{i:05d}' for i in np.random.randint(1, 1001, N)],
    '지역':     np.random.choice(regions, N),
    '상품':     np.random.choice(products, N),
    '판매수량': np.random.randint(1, 50, N),
    '단가':     np.random.choice([50000, 120000, 80000, 350000, 45000], N),
})
df_big['매출액'] = df_big['판매수량'] * df_big['단가']

df_big.to_csv('sales_big.csv', index=False, encoding='utf-8-sig')
print(f'✅ 더미 데이터 생성: {N:,}행 → sales_big.csv')
print(f'📦 파일 크기: {Path("sales_big.csv").stat().st_size / 1024 / 1024:.1f} MB')

In [ ]:
# ─── 방법 1: 한 번에 전체 읽기 (메모리 위험) ───────────────
import time

t0 = time.time()
df_all = pd.read_csv('sales_big.csv', encoding='utf-8-sig')
t1 = time.time()

mem_mb = df_all.memory_usage(deep=True).sum() / 1024 / 1024
print(f'전체 읽기: {t1-t0:.2f}초, 메모리: {mem_mb:.1f} MB, 행수: {len(df_all):,}')

In [ ]:
# ─── 방법 2: dtype 최적화로 메모리 절약 ──────────────────────
dtype_map = {
    '부서':     'category',   # 반복 문자열 → category로 압축
    '지역':     'category',
    '상품':     'category',
    '판매수량': 'int16',      # int64 → int16 (최대 32767)
    '단가':     'int32',
    '매출액':   'int64',
}

t0 = time.time()
df_opt = pd.read_csv('sales_big.csv', encoding='utf-8-sig',
                     dtype=dtype_map,
                     usecols=['날짜','부서','지역','상품','판매수량','단가','매출액'])
t1 = time.time()

mem_opt = df_opt.memory_usage(deep=True).sum() / 1024 / 1024
print(f'최적화 읽기: {t1-t0:.2f}초, 메모리: {mem_opt:.1f} MB')
print(f'💡 메모리 절약: {mem_mb - mem_opt:.1f} MB ({(1 - mem_opt/mem_mb)*100:.0f}% 감소)')

In [ ]:
# ─── 방법 3: 청크 처리 (메모리 한계 초과할 때) ───────────────
# 한 번에 10,000행씩 나눠 읽어서 집계

CHUNK_SIZE = 10_000
results = []   # 각 청크의 집계 결과 저장

t0 = time.time()
for i, chunk in enumerate(pd.read_csv('sales_big.csv', encoding='utf-8-sig',
                                       chunksize=CHUNK_SIZE)):
    # 각 청크에서 부서별 매출 집계
    agg = chunk.groupby('부서')['매출액'].sum()
    results.append(agg)

# 모든 청크 결과를 합산
df_chunk_result = pd.concat(results).groupby(level=0).sum()
t1 = time.time()

print(f'청크 처리: {t1-t0:.2f}초 (청크 {i+1}개 처리)')
print('\n📊 부서별 총 매출:')
for dept, sales in df_chunk_result.sort_values(ascending=False).items():
    print(f'  {dept}: {sales:,.0f}원')

---
## 2️⃣ 멀티파일 자동 통합

> **여러 Excel/CSV 파일**을 자동으로 찾아서 하나로 합치는 패턴

- `glob` / `pathlib.Path.glob()` 으로 파일 탐색
- 파일명에서 메타정보(월, 지점명 등) 추출
- 오류 파일 건너뛰기 + 로그 기록

In [ ]:
# 실습: 월별 파일 5개 생성
import os

os.makedirs('monthly_data', exist_ok=True)

months = ['2024-01', '2024-02', '2024-03', '2024-04', '2024-05']
for month in months:
    n = random.randint(80, 200)
    df_m = pd.DataFrame({
        '날짜':     [month] * n,
        '부서':     np.random.choice(departments, n),
        '상품':     np.random.choice(products, n),
        '판매수량': np.random.randint(1, 30, n),
        '매출액':   np.random.randint(100000, 5000000, n),
    })
    path = f'monthly_data/sales_{month}.xlsx'
    df_m.to_excel(path, index=False)
    print(f'  ✅ {path} ({n}행)')

print('\n✅ 월별 파일 5개 생성 완료')

In [ ]:
# 멀티파일 통합 함수
from pathlib import Path

def merge_excel_files(folder: str, pattern: str = '*.xlsx') -> pd.DataFrame:
    """
    폴더 내 모든 Excel 파일을 하나의 DataFrame으로 통합
    - 파일명에서 '월' 정보를 자동 추출
    - 오류 파일은 건너뛰고 계속 진행
    """
    files = sorted(Path(folder).glob(pattern))
    print(f'📁 발견된 파일: {len(files)}개')

    frames = []
    errors = []

    for f in files:
        try:
            df_tmp = pd.read_excel(f)
            # 파일명에서 월 추출: sales_2024-01.xlsx → 2024-01
            month = f.stem.replace('sales_', '')
            df_tmp['출처파일'] = month
            frames.append(df_tmp)
            print(f'  ✅ {f.name}: {len(df_tmp):,}행')
        except Exception as e:
            errors.append(f.name)
            print(f'  ❌ {f.name}: {e}')

    result = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    print(f'\n📊 통합 결과: {len(result):,}행, 오류 {len(errors)}개')
    return result


df_merged = merge_excel_files('monthly_data')
print(df_merged.head(3))

---
## 3️⃣ 동적 대시보드 생성

> **xlwings로 실시간 데이터** + **openpyxl로 정밀 서식** = 프로급 대시보드

```
[대시보드 구성]
┌──────────────────────────────────────────┐
│  KPI 요약 카드  │  월별 추이 차트         │
├──────────────────────────────────────────┤
│  부서별 실적 표 │  상품별 매출 파이차트   │
└──────────────────────────────────────────┘
```

In [ ]:
# openpyxl 기반 대시보드 생성
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.chart import BarChart, LineChart, PieChart, Reference

import pandas as pd
import numpy as np
import random

# 집계 데이터 준비
dept_summary = df_merged.groupby('부서').agg(
    건수=('매출액', 'count'),
    총매출=('매출액', 'sum'),
    평균매출=('매출액', 'mean')
).round(0).reset_index()

monthly_summary = df_merged.groupby('출처파일')['매출액'].sum().reset_index()
monthly_summary.columns = ['월', '매출액']

print('부서별 요약:')
print(dept_summary)
print('\n월별 요약:')
print(monthly_summary)

In [ ]:
# 대시보드 Excel 파일 생성
wb_dash = Workbook()
ws = wb_dash.active
ws.title = '대시보드'

thin = Side(style='thin')
border = Border(top=thin, bottom=thin, left=thin, right=thin)

# ── KPI 타이틀 ──────────────────────────────────────────
ws['A1'] = '📊 월간 영업 대시보드'
ws['A1'].font = Font(name='맑은 고딕', size=18, bold=True, color='2F5597')
ws['A1'].alignment = Alignment(horizontal='center', vertical='center')
ws.merge_cells('A1:K1')
ws.row_dimensions[1].height = 45

# ── KPI 카드 (A3:K5) ────────────────────────────────────
total_sales  = df_merged['매출액'].sum()
total_count  = len(df_merged)
avg_sales    = df_merged['매출액'].mean()
top_dept     = dept_summary.loc[dept_summary['총매출'].idxmax(), '부서']

kpi_data = [
    ('💰 총 매출', f'{total_sales:,.0f}원',  'C6EFCE', '006100'),
    ('📋 총 건수', f'{total_count:,}건',     'DDEEFF', '003399'),
    ('📈 평균 매출', f'{avg_sales:,.0f}원',  'FFF2CC', '7D6608'),
    ('🏆 최고 부서', top_dept,               'FCE4D6', '843C0C'),
]

kpi_cols = [1, 4, 7, 10]  # A, D, G, J 열
for (label, value, bg, fg), col in zip(kpi_data, kpi_cols):
    # 제목
    c_title = ws.cell(row=3, column=col, value=label)
    c_title.font = Font(name='맑은 고딕', bold=True, color='595959', size=10)
    c_title.fill = PatternFill(fill_type='solid', fgColor=bg)
    c_title.alignment = Alignment(horizontal='center', vertical='center')
    ws.merge_cells(start_row=3, start_column=col, end_row=3, end_column=col+2)
    # 값
    c_val = ws.cell(row=4, column=col, value=value)
    c_val.font = Font(name='맑은 고딕', bold=True, color=fg, size=13)
    c_val.fill = PatternFill(fill_type='solid', fgColor=bg)
    c_val.alignment = Alignment(horizontal='center', vertical='center')
    ws.merge_cells(start_row=4, start_column=col, end_row=5, end_column=col+2)

ws.row_dimensions[3].height = 20
ws.row_dimensions[4].height = 35
ws.row_dimensions[5].height = 10

# ── 부서별 데이터 표 (A7~) ──────────────────────────────
START_ROW = 7
headers = ['부서', '건수', '총매출', '평균매출']
h_fill  = PatternFill(fill_type='solid', fgColor='2F5597')
h_font  = Font(name='맑은 고딕', bold=True, color='FFFFFF')
h_align = Alignment(horizontal='center', vertical='center')

for ci, h in enumerate(headers, 1):
    cell = ws.cell(row=START_ROW, column=ci, value=h)
    cell.font = h_font; cell.fill = h_fill
    cell.alignment = h_align; cell.border = border

even_fill = PatternFill(fill_type='solid', fgColor='DCE6F1')
for ri, (_, row) in enumerate(dept_summary.iterrows(), START_ROW+1):
    rf = even_fill if ri % 2 == 0 else None
    for ci, val in enumerate(row, 1):
        c = ws.cell(row=ri, column=ci, value=val)
        c.border = border
        c.alignment = Alignment(horizontal='center' if ci > 1 else 'left',
                                 vertical='center')
        if rf: c.fill = rf
        if ci in (3, 4):
            c.number_format = '#,##0'

col_widths = {'A':12,'B':8,'C':16,'D':14}
for col, w in col_widths.items():
    ws.column_dimensions[col].width = w

print('✅ KPI 카드 + 부서별 표 완료')

In [ ]:
# ── 차트 추가 ────────────────────────────────────────────

# 월별 추이 데이터를 차트용 시트에 기록
ws_chart_data = wb_dash.create_sheet('차트데이터')
ws_chart_data['A1'] = '월'
ws_chart_data['B1'] = '매출액'
for ri, (_, row) in enumerate(monthly_summary.iterrows(), 2):
    ws_chart_data.cell(row=ri, column=1, value=row['월'])
    ws_chart_data.cell(row=ri, column=2, value=row['매출액'])

# 월별 라인 차트
lc = LineChart()
lc.title  = '월별 매출 추이'
lc.y_axis.title = '매출액(원)'
lc.x_axis.title = '월'
lc.style  = 10
lc.width  = 18
lc.height = 12

n_months = len(monthly_summary)
data_ref = Reference(ws_chart_data, min_col=2, min_row=1, max_row=n_months+1)
cats_ref = Reference(ws_chart_data, min_col=1, min_row=2, max_row=n_months+1)
lc.add_data(data_ref, titles_from_data=True)
lc.set_categories(cats_ref)
ws['F3'] = ''
ws.add_chart(lc, 'F3')  # 대시보드 F3 위치에 삽입

wb_dash.save('dashboard.xlsx')
print('✅ 대시보드 저장 완료: dashboard.xlsx')
print('💡 파일을 열어 대시보드를 확인하세요!')

---
## 4️⃣ 예외 처리 & 로깅

> 실무 자동화에서 **오류가 나도 멈추지 않고** 로그를 남기는 방법

| 상황 | 해결책 |
|:---|:---|
| 파일 없음 | `FileNotFoundError` 처리 |
| 빈 시트 | None 체크 후 스킵 |
| 형식 불일치 | 컬럼 존재 여부 검증 |
| 전체 로그 | `logging` 모듈 파일 기록 |

In [ ]:
import logging
from datetime import datetime

# 로거 설정 (파일 + 콘솔 동시 출력)
def setup_logger(log_file: str = 'automation.log') -> logging.Logger:
    logger = logging.getLogger('ExcelAuto')
    logger.setLevel(logging.DEBUG)

    fmt = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s',
                             datefmt='%Y-%m-%d %H:%M:%S')

    # 파일 핸들러
    fh = logging.FileHandler(log_file, encoding='utf-8')
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(fmt)

    # 콘솔 핸들러
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    ch.setFormatter(fmt)

    if not logger.handlers:
        logger.addHandler(fh)
        logger.addHandler(ch)

    return logger


logger = setup_logger()
logger.info('=== 자동화 시작 ===')
logger.info('로거 설정 완료')
print('✅ 로거 설정 완료 → automation.log 에 기록됩니다')

In [ ]:
# 예외 처리가 포함된 안전한 파일 읽기 함수

REQUIRED_COLS = {'부서', '매출액', '판매수량'}

def safe_read_excel(path: str) -> pd.DataFrame | None:
    """
    Excel 파일을 안전하게 읽는 함수
    - 파일 없음, 빈 파일, 컬럼 불일치 모두 처리
    - 문제 발생 시 None 반환 + 로그 기록
    """
    try:
        if not Path(path).exists():
            raise FileNotFoundError(f'파일 없음: {path}')

        df = pd.read_excel(path)

        if df.empty:
            logger.warning(f'빈 파일 스킵: {path}')
            return None

        missing = REQUIRED_COLS - set(df.columns)
        if missing:
            raise ValueError(f'필수 컬럼 없음: {missing}')

        logger.info(f'읽기 성공: {path} ({len(df):,}행)')
        return df

    except FileNotFoundError as e:
        logger.error(f'FileNotFoundError: {e}')
    except ValueError as e:
        logger.error(f'ValueError ({path}): {e}')
    except Exception as e:
        logger.exception(f'알 수 없는 오류 ({path}): {e}')

    return None


# 테스트
r1 = safe_read_excel('monthly_data/sales_2024-01.xlsx')
r2 = safe_read_excel('존재하지않는파일.xlsx')  # 오류 케이스

print(f'\n✅ 성공: {r1 is not None}, 실패: {r2 is None}')

---
## 5️⃣ 이메일 자동 발송

> 생성된 보고서를 **이메일로 자동 발송**하는 방법

⚠️ **주의**: Gmail 사용 시 `앱 비밀번호`가 필요합니다  
(Google 계정 → 보안 → 2단계 인증 → 앱 비밀번호 생성)

```
이메일 발송 흐름
────────────────
1. MIMEMultipart 메시지 객체 생성
2. 제목/본문 설정 (HTML 형식 가능)
3. Excel 파일 첨부 (MIMEBase)
4. SMTP 서버 연결 → SSL 인증
5. sendmail() 발송
```

In [ ]:
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from datetime import date

def send_report_email(
    smtp_user: str,
    smtp_pass: str,
    to_addr: str,
    attach_path: str,
    smtp_host: str = 'smtp.gmail.com',
    smtp_port: int = 465
):
    """
    Excel 보고서를 이메일로 자동 발송
    Parameters:
        smtp_user  : 발신자 Gmail 주소
        smtp_pass  : Gmail 앱 비밀번호 (16자리)
        to_addr    : 수신자 이메일
        attach_path: 첨부할 Excel 파일 경로
    """
    today = date.today().strftime('%Y년 %m월 %d일')

    msg = MIMEMultipart('alternative')
    msg['Subject'] = f'[자동발송] 월간 영업 보고서 - {today}'
    msg['From']    = smtp_user
    msg['To']      = to_addr

    # HTML 본문
    html_body = f"""
    <html><body>
    <h2>📊 월간 영업 보고서</h2>
    <p>안녕하세요,</p>
    <p>{today} 기준 월간 영업 보고서를 첨부합니다.</p>
    <ul>
      <li>총 매출: <b>{df_merged['매출액'].sum():,.0f}원</b></li>
      <li>총 건수: <b>{len(df_merged):,}건</b></li>
    </ul>
    <p>감사합니다.</p>
    </body></html>
    """
    msg.attach(MIMEText(html_body, 'html', 'utf-8'))

    # 파일 첨부
    if os.path.exists(attach_path):
        with open(attach_path, 'rb') as f:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(f.read())
        encoders.encode_base64(part)
        fname = os.path.basename(attach_path)
        part.add_header('Content-Disposition', f'attachment; filename="{fname}"')
        msg.attach(part)
        logger.info(f'첨부 파일: {fname}')

    # 발송
    with smtplib.SMTP_SSL(smtp_host, smtp_port) as server:
        server.login(smtp_user, smtp_pass)
        server.sendmail(smtp_user, to_addr, msg.as_string())

    logger.info(f'이메일 발송 완료 → {to_addr}')
    print(f'✅ 이메일 발송 완료: {to_addr}')


# ────────────────────────────────────────────────────────
# 실제 발송 시 아래 주석을 해제하고 정보를 입력하세요
# ────────────────────────────────────────────────────────
# send_report_email(
#     smtp_user   = 'your_gmail@gmail.com',
#     smtp_pass   = 'xxxx xxxx xxxx xxxx',  # 앱 비밀번호 16자리
#     to_addr     = 'recipient@example.com',
#     attach_path = 'dashboard.xlsx'
# )
print('💡 위 주석을 해제하고 본인 이메일 정보를 입력하면 실제 발송됩니다')

---
## 🔥 6️⃣ 실전 종합 프로젝트: 월간 영업 보고서 자동화 시스템

**시나리오**: 매월 1일 오전 9시, 다음 작업이 자동으로 실행됩니다:

```
① monthly_data/ 폴더에서 지난달 파일 자동 탐색
② 전체 파일 통합 (merge_excel_files)
③ 대시보드 Excel 생성 (KPI 카드 + 차트)
④ 이메일 자동 발송 (관리자에게)
⑤ 로그 기록 (automation.log)
```

모든 과정을 하나의 함수로 캡슐화합니다!

In [ ]:
def run_monthly_pipeline(
    data_folder: str = 'monthly_data',
    output_path: str = 'monthly_report_final.xlsx',
    send_email: bool = False,
    email_config: dict = None
):
    """
    🔥 월간 영업 보고서 자동화 파이프라인

    Parameters:
        data_folder : 월별 Excel 파일이 있는 폴더
        output_path : 생성할 보고서 경로
        send_email  : True면 이메일 자동 발송
        email_config: {'user', 'pass', 'to'} 딕셔너리
    """
    logger.info('=' * 50)
    logger.info('월간 보고서 자동화 파이프라인 시작')

    # STEP 1: 멀티파일 통합
    logger.info('[STEP 1] 파일 통합 시작')
    df = merge_excel_files(data_folder)
    if df.empty:
        logger.error('통합 데이터 없음 → 파이프라인 중단')
        return
    logger.info(f'[STEP 1] 완료: 총 {len(df):,}행')

    # STEP 2: 집계
    logger.info('[STEP 2] 데이터 집계')
    dept_agg = df.groupby('부서').agg(
        건수=('매출액', 'count'),
        총매출=('매출액', 'sum'),
        평균매출=('매출액', 'mean')
    ).round(0).reset_index()

    month_agg = df.groupby('출처파일')['매출액'].sum().reset_index()
    month_agg.columns = ['월', '매출액']

    # STEP 3: 보고서 생성
    logger.info('[STEP 3] 보고서 생성')
    wb_final = Workbook()

    # ── 표지 시트 ────────────────────────────────────────
    ws_cover = wb_final.active
    ws_cover.title = '표지'
    ws_cover.column_dimensions['A'].width = 50
    ws_cover.row_dimensions[8].height = 60
    ws_cover['A8'] = '📊 월간 영업 보고서 (자동 생성)'
    ws_cover['A8'].font = Font(name='맑은 고딕', size=22, bold=True, color='2F5597')
    ws_cover['A8'].alignment = Alignment(horizontal='center', vertical='center')
    from datetime import date
    ws_cover['A10'] = f'생성일시: {date.today().strftime("%Y년 %m월 %d일")}'
    ws_cover['A10'].font = Font(name='맑은 고딕', size=13, color='595959')
    ws_cover['A10'].alignment = Alignment(horizontal='center')
    ws_cover['A12'] = f'대상 기간: {month_agg["월"].min()} ~ {month_agg["월"].max()}'
    ws_cover['A12'].font = Font(name='맑은 고딕', size=12, color='595959')
    ws_cover['A12'].alignment = Alignment(horizontal='center')

    # ── 데이터 시트 ──────────────────────────────────────
    ws_data = wb_final.create_sheet('원본데이터')
    thin = Side(style='thin')
    border = Border(top=thin, bottom=thin, left=thin, right=thin)
    h_fill = PatternFill(fill_type='solid', fgColor='2F5597')
    h_font = Font(name='맑은 고딕', bold=True, color='FFFFFF')
    h_align = Alignment(horizontal='center', vertical='center')

    cols = list(df.columns)
    for ci, col in enumerate(cols, 1):
        c = ws_data.cell(row=1, column=ci, value=col)
        c.font = h_font; c.fill = h_fill
        c.alignment = h_align; c.border = border
        ws_data.column_dimensions[get_column_letter(ci)].width = 14

    even_fill = PatternFill(fill_type='solid', fgColor='DCE6F1')
    for ri, (_, row) in enumerate(df.iterrows(), 2):
        rf = even_fill if ri % 2 == 0 else None
        for ci, val in enumerate(row, 1):
            c = ws_data.cell(row=ri, column=ci, value=val)
            c.border = border
            if rf: c.fill = rf

    # ── 부서요약 시트 ─────────────────────────────────────
    ws_dept = wb_final.create_sheet('부서요약')
    for ci, col in enumerate(dept_agg.columns, 1):
        c = ws_dept.cell(row=1, column=ci, value=col)
        c.font = h_font; c.fill = h_fill
        c.alignment = h_align; c.border = border
        ws_dept.column_dimensions[get_column_letter(ci)].width = 16
    for ri, (_, row) in enumerate(dept_agg.iterrows(), 2):
        for ci, val in enumerate(row, 1):
            c = ws_dept.cell(row=ri, column=ci, value=val)
            c.border = border
            c.alignment = Alignment(horizontal='center', vertical='center')
            if ci in (3, 4): c.number_format = '#,##0'

    # ── 차트 시트 ─────────────────────────────────────────
    ws_cd = wb_final.create_sheet('차트데이터')
    ws_cd['A1'] = '월'; ws_cd['B1'] = '매출액'
    for ri, (_, row) in enumerate(month_agg.iterrows(), 2):
        ws_cd.cell(row=ri, column=1, value=row['월'])
        ws_cd.cell(row=ri, column=2, value=row['매출액'])

    bc = BarChart()
    bc.title  = '월별 매출 현황'
    bc.y_axis.title = '매출액'
    bc.style  = 10; bc.width = 18; bc.height = 12
    nm = len(month_agg)
    bc.add_data(Reference(ws_cd, min_col=2, min_row=1, max_row=nm+1), titles_from_data=True)
    bc.set_categories(Reference(ws_cd, min_col=1, min_row=2, max_row=nm+1))
    ws_dept.add_chart(bc, 'G2')

    # STEP 4: 저장
    wb_final.save(output_path)
    logger.info(f'[STEP 4] 보고서 저장: {output_path}')

    # STEP 5: 이메일 발송 (선택)
    if send_email and email_config:
        send_report_email(
            smtp_user=email_config['user'],
            smtp_pass=email_config['pass'],
            to_addr=email_config['to'],
            attach_path=output_path
        )

    logger.info('파이프라인 완료!')
    print(f'\n🎉 파이프라인 완료! → {output_path}')
    return df


# ── 실행 ─────────────────────────────────────────────────
result_df = run_monthly_pipeline(
    data_folder = 'monthly_data',
    output_path = 'monthly_report_final.xlsx',
    send_email  = False   # True + email_config 설정 시 발송
)

---
## ⏰ 스케줄링: 매월 자동 실행

### 방법 A: Python schedule 라이브러리

```python
import schedule, time

# 매월 1일 오전 9시 실행
schedule.every().month.at('09:00').do(run_monthly_pipeline)

while True:
    schedule.run_pending()
    time.sleep(3600)  # 1시간마다 체크
```

### 방법 B: Windows 작업 스케줄러 (추천 ⭐)

```cmd
# 매월 1일 09:00 실행 등록
schtasks /create /tn "월간보고서자동화" /tr "python e:\excelAuto\run_pipeline.py" /sc monthly /d 1 /st 09:00
```

> 💡 **Windows 작업 스케줄러**를 사용하면 Python이 실행 중이 아니어도 자동으로 실행됩니다!

---
## ⚠️ 흔한 에러와 해결

| 증상 | 원인 | 해결 |
|:---|:---|:---|
| `MemoryError` | 대용량 파일 전체 읽기 | 청크 처리 또는 dtype 최적화 |
| `EmptyDataError` | 빈 CSV/Excel | 읽기 전 파일 크기/행수 체크 |
| SMTP 인증 실패 | 일반 비밀번호 사용 | Gmail 앱 비밀번호 16자리 사용 |
| 차트가 빈 화면 | 데이터 참조 범위 오류 | `min_row=1` (헤더 포함) 확인 |
| `COMError` | Excel이 열려 있음 | 작업관리자에서 Excel 프로세스 종료 |
| 한글 파일명 깨짐 | 인코딩 문제 | `encoding='utf-8-sig'` 지정 |
| 경로 오류 (Windows) | 역슬래시 문제 | `Path()` 사용 또는 `r'경로'` 사용 |

---
## 📝 심화 연습 문제

1. **대용량 처리**: `sales_big.csv`에서 **지역별 + 상품별** 매출 합계를 청크 방식으로 계산하세요
2. **멀티파일**: `monthly_data/` 에 파일을 추가로 생성하고 통합 함수를 실행해 보세요
3. **대시보드 확장**: KPI 카드에 **전월 대비 증감률(%)**을 추가해 보세요
4. **로깅 활용**: `safe_read_excel()` 에서 읽은 파일의 **행수/열수를 항상 로그에 기록**하도록 개선하세요
5. **종합 도전**: `run_monthly_pipeline()`을 수정하여 **상품별 요약 시트**를 추가하세요

In [ ]:
# 🏋️ 여기에 연습 문제를 풀어보세요!

